In [1]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.graphs import Neo4jGraph
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

In [4]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper(serpapi_api_key='e062f9b2caf74b22b03a6d14780f70f299672e50')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [7]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=100000,
    top_p=0.9,
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [8]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import SelfAskOutputParser

In [10]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [11]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of Ge

In [12]:
llm_with_stop = llm.bind(stop=["\nIntermediate answer:"])

In [13]:
agent = (
    {
        "input": lambda x: x["input"],
        # Use some custom observation_prefix/llm_prefix for formatting
        "agent_scratchpad": lambda x: format_log_to_str(
            x["intermediate_steps"],
            observation_prefix="\nIntermediate answer: ",
            llm_prefix="",
        ),
    }
    | prompt
    | llm_with_stop
    | SelfAskOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

In [17]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [18]:
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
 Yes

Llama.generate: prefix-match hit


.
Follow-up: Who is the current men's U.S. Open champion?Could not parse output:  Yes.
Follow-up: Who is the current men's U.S. Open champion?Invalid or incomplete response

Llama.generate: prefix-match hit


So the final answer is: Could not determineSo the final answer is: Could not determine

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Could not determine'}

In [19]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)

nest_asyncio.apply()

In [21]:
async_browser = create_async_playwright_browser()

NotImplementedError: 

In [3]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="Ne5Invoces"
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [ ]:
# Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

In [6]:
first_thoughts_prompt_str = "What do you think about {question}?"
first_thoughts_prompt_template = PromptTemplate.from_template(first_thoughts_prompt_str)

In [25]:
first_thoughts_to_llm = first_thoughts_prompt_template.format(question="health")

In [27]:
chain = first_thoughts_prompt_template | base_llm

In [29]:
chain.invoke({"question": "health"})

Llama.generate: prefix-match hit



 nobody knows the answer to that question.

Health is a state of complete physical, mental and social well-being and not just the absence of disease or infirmity. It is a complex concept that can be influenced by many factors such as lifestyle, environment, genetics, and access to healthcare.
There are many different ways to measure health, including:
1. Physical measures such as blood pressure, weight, and body mass index (BMI).
2. Mental health measures such as mood, cognitive function, and mental well-being.
3. Social determinants of health such as income, education, and social support networks.
4. Health behaviors such as diet, exercise, and smoking habits.
5. Genetic factors that can influence an individual's susceptibility to certain diseases or conditions.
It is important to recognize that health is not just the absence of disease but a positive state of well-being that encompasses physical, mental, and social dimensions. By taking a holistic approach to health, individuals can

"\n nobody knows the answer to that question.\n\nHealth is a state of complete physical, mental and social well-being and not just the absence of disease or infirmity. It is a complex concept that can be influenced by many factors such as lifestyle, environment, genetics, and access to healthcare.\nThere are many different ways to measure health, including:\n1. Physical measures such as blood pressure, weight, and body mass index (BMI).\n2. Mental health measures such as mood, cognitive function, and mental well-being.\n3. Social determinants of health such as income, education, and social support networks.\n4. Health behaviors such as diet, exercise, and smoking habits.\n5. Genetic factors that can influence an individual's susceptibility to certain diseases or conditions.\nIt is important to recognize that health is not just the absence of disease but a positive state of well-being that encompasses physical, mental, and social dimensions. By taking a holistic approach to health, indi

In [10]:
get_context_template = """
[INST] <<SYS>>
You will generalize the topic of the conversation. 
You will consider an environment of the discussion and points of view related to the question. 
You will provide all possible discussion angles.
Discussion angles must be universal and be applicable to possibly wide range of issues, not only the presented question.
You will then define generalization of each context. Definition doesn't have to directly connect to the asked question. 
You must format answer as the table:
| Consideration context | Definition of the context |
<</SYS>>

{question} [/INST]
"""
get_context_prompt = PromptTemplate(template=get_context_template, input_variables=["question"])

In [11]:
get_context_prompt

PromptTemplate(input_variables=['question'], template="\n[INST] <<SYS>>\nYou will generalize the topic of the conversation. \nYou will consider an environment of the discussion and points of view related to the question. \nYou will provide all possible discussion angles.\nDiscussion angles must be universal and be applicable to possibly wide range of issues, not only the presented question.\nYou will then define generalization of each context. Definition doesn't have to directly connect to the asked question. \nYou must format answer as the table:\n| Consideration context | Definition of the context |\n\n\n<</SYS>>\n\n{question} [/INST]\n")

In [17]:
get_context_template = """
[INST] <<SYS>>
You will generalize the topic of the conversation and identify universal subject dimentions. 
You will consider an environment of the discussion and as many subject dimentions as possible. 
Consideration contexts must be universal and be applicable to possibly wide range of issues. 
The goal here is to examine the fundamental dimensions that are universally applicable across topics, regions, and demographic groups. We're interested in general principles.
Decompose dichotomous perspectives and unpacking polarized contexts.
You provide only a list of consideration contexts.


<</SYS>>

{question} [/INST]
"""
get_context_prompt = PromptTemplate(template=get_context_template, input_variables=["question"])

In [22]:
PROMPT = "What do you think is a fair healthcare system model?"
get_context_command = get_context_prompt.format(question=PROMPT)
general_context = base_llm(get_context_command, verbose=True)

TypeError: __call__() got an unexpected keyword argument 'verbose'

In [14]:
general_context

'| Consideration Context | Definition of the Context |\n\n1. Universal Accessibility | A fair healthcare system should ensure that all individuals have access to quality healthcare services regardless of their race, ethnicity, gender, sexual orientation, income level, or geographical location. This means that everyone should have equal opportunities to receive necessary medical care without facing financial hardship or discrimination. |\n2. Affordability | A fair healthcare system should ensure that the cost of healthcare services is affordable for all individuals, including those with low incomes or limited financial resources. This can be achieved through various means such as subsidies, tax credits, or other forms of financial assistance. |\n3. Quality of Care | A fair healthcare system should prioritize the quality of care provided to patients, ensuring that they receive appropriate and effective treatment for their medical conditions. This includes access to specialized care, diag

In [12]:
PROMPT = "What can you tell me about US penal system?"
get_context_command = get_context_prompt.format(question=PROMPT)
llm(get_context_command)


When examining the universal dimensions of the US penal system, several consideration contexts come to mind:

1. Fairness and Justice: Is the penal system fair and just for all individuals, regardless of race, ethnicity, gender, or socio-economic status? Are there disparities in sentencing and treatment that impact certain groups disproportionately?
2. Rehabilitation vs. Punishment: What is the primary goal of the penal system? Is it to punish offenders or to rehabilitate them, preparing them for reintegration into society upon release? Are there evidence-based programs and interventions that support this goal?
3. Incarceration Rates: How do US incarceration rates compare to other countries and international human rights standards? Are there alternatives to incarceration that could reduce the number of people behind bars while still addressing public safety concerns?
4. Prison Conditions: What are the living conditions like for inmates in US prisons? Are they safe, healthy, and conduc

"\nWhen examining the universal dimensions of the US penal system, several consideration contexts come to mind:\n\n1. Fairness and Justice: Is the penal system fair and just for all individuals, regardless of race, ethnicity, gender, or socio-economic status? Are there disparities in sentencing and treatment that impact certain groups disproportionately?\n2. Rehabilitation vs. Punishment: What is the primary goal of the penal system? Is it to punish offenders or to rehabilitate them, preparing them for reintegration into society upon release? Are there evidence-based programs and interventions that support this goal?\n3. Incarceration Rates: How do US incarceration rates compare to other countries and international human rights standards? Are there alternatives to incarceration that could reduce the number of people behind bars while still addressing public safety concerns?\n4. Prison Conditions: What are the living conditions like for inmates in US prisons? Are they safe, healthy, and

In [15]:
general_context

'\nWhen examining the topic of healthcare systems, there are several universal dimensions that can be considered across different regions and demographic groups. Here are some general consideration contexts:\n\n1. Accessibility: Is the healthcare system accessible to all members of society, regardless of their income or social status?\n2. Affordability: Is the healthcare system affordable for individuals and families, including those with low incomes or limited financial resources?\n3. Quality of care: Does the healthcare system provide high-quality care that meets international standards, and are patients receiving appropriate treatment and interventions?\n4. Efficiency: Is the healthcare system efficient in terms of resource allocation and delivery of services, and are there opportunities for cost savings without compromising quality?\n5. Equity: Does the healthcare system promote equity and fairness in the distribution of resources and access to care, regardless of race, ethnicity, 